In [ ]:
# Import useful libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
# Sobel Filter Function
Sobel_X = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
Sobel_Y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=np.float32)

# Gaussian Filter Function
Gaussian = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]], dtype=np.float32) / 16

In [ ]:
# Convolution Function
def Convolution(image, kernel):
    # Get the dimensions of the image and kernel
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape

    # Create an copy image to store the output
    # Done to prevent the edges from being affected due to convolution
    new_image = np.copy(image).astype(np.float32)

    # Initialize useful variables
    start_y = kernel_height // 2
    start_x = kernel_width // 2
    end_y = image_height - kernel_height // 2
    end_x = image_width - kernel_width // 2

    # Convolve the image with the kernel
    for y in range(start_y, end_y):
        for x in range(start_x, end_x):
            # Get the region of the image
            region = image[y - start_y:y + start_y + 1, x - start_x:x + start_x + 1]

            # Perform the convolution
            new_image[y, x] = np.sum(region * kernel)

    return new_image

In [ ]:
def getCornerScratch(I_xx, I_yy, I_xy, k):
    # Get the dimensions of the image
    height, width = I_xx.shape

    # Initialize the corner response image
    harris_response = np.zeros((height, width), dtype=np.float32)

    # Compute the corner response
    for y in range(height):
        for x in range(width):
            M = np.array([[I_xx[y, x], I_xy[y, x]], [I_xy[y, x],I_yy[y, x]]], dtype=np.float32)
            R = np.linalg.det(M) - k * (np.trace(M) ** 2)
            harris_response[y, x] = R

    return harris_response

In [ ]:
def getCornerFromHarrisResponse(R, threshold, window_size):
    # Get the dimensions of the image
    height, width = R.shape

    # Initialize the corner image
    corner_image = np.zeros((height, width), dtype=np.float32)

    # Initialize useful variables
    start_y = window_size // 2
    start_x = window_size // 2
    end_y = height - window_size // 2
    end_x = width - window_size // 2

    # Get the corners from the Harris Response
    for y in range(start_y, end_y):
        for x in range(start_x, end_x):
            # Get the region of the Harris Response
            region = R[y - start_y:y + start_y + 1, x - start_x:x + start_x + 1]
            if R[y, x] > threshold and R[y, x] == np.max(region):
                corner_image[y, x] = 1

    return corner_image

In [ ]:
image_path = []

for path in image_path:

    img = cv2.imread(path)

    colored_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Sobel Filter
    I_X = Convolution(gray_img, Sobel_X)
    I_Y = Convolution(gray_img, Sobel_Y)

    I_xx = np.square(I_X)
    I_yy = np.square(I_Y)
    I_xy = I_X * I_Y

    I_xx_Blur = Convolution(I_xx, Gaussian)
    I_yy_Blur = Convolution(I_yy, Gaussian)
    I_xy_Blur = Convolution(I_xy, Gaussian)
    
    # Compute the Harris Corner Response
    k = 0.04
    window_size = 5
    R = getCornerScratch(I_xx_Blur, I_yy_Blur, I_xy_Blur, k)
    C_Response = getCornerFromHarrisResponse(R, 0.2*np.max(R), window_size)
    
    # Get the corners
    corners = np.argwhere(C_Response == 1)
    
    # Corner Detection using OpenCV
    gray_img = np.float32(gray_img)

    dst = cv2.cornerHarris(gray_img, 5, 3, 0.04)
    dst = cv2.dilate(dst, None)

    threshold = 0.20 * dst.max()
    
    corner_img = np.copy(colored_img)

    for j in range(dst.shape[0]):
        for i in range(dst.shape[1]):
            if dst[j, i] > threshold:
                cv2.circle(corner_img, (i, j), 1, (255, 0, 0), -1)
    
    # Display 2 images in horizontal line
    fig, axs = plt.subplots(1, 2, figsize=(14, 7))
    axs[0].imshow(corner_img)
    axs[0].set_title('Using OpenCV')
    axs[0].axis('off')
    axs[1].imshow(colored_img)
    axs[1].scatter(corners[:, 1], corners[:, 0], c='red', s=10)
    axs[1].set_title('Using Scratch Implementation')
    axs[1].axis('off')
    plt.show()